# Bandgap Prediction

Using QM9 data set to predict the bandgap of molecules using LSTM and GNN's

In [14]:
from torch_geometric.datasets import QM9
from torch_geometric.loader import DataLoader as GraphDataLoader
import torch
import itertools
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd

In [34]:
def load_qm9(path="./QM9"):
    return QM9(path)

qm9 = load_qm9()
smiles = []
coords = []
homo_lumo_gaps = []
stock_gap = []
homo = []
lumo = []

for entry in qm9:
    smiles.append(entry.smiles)  # Get SMILES notation
    coords.append(entry.pos)     # Get atomic coordinates
    homo_energy = entry.y[0, 2].item()  # Get HOMO energy from the tensor
    homo.append(homo_energy)
    lumo_energy = entry.y[0, 3].item()  # Get LUMO energy from the tensor
    lumo.append(lumo_energy)
    gap_stock = entry.y[0, 4].item() 
    gap = lumo_energy - homo_energy  # Calculate the bandgap
    stock_gap.append(gap_stock)
    homo_lumo_gaps.append(gap)  # Store the bandgap
# Creating a DataFrame for better visualization and handling
data = pd.DataFrame({
    "SMILES": smiles,
    "Coordinates": coords,
    "HOMO" : homo,
    "LUMO" : lumo,
    "Egap" : stock_gap
})

# Print the first few rows of the DataFrame to verify
print(data.head())

              SMILES                                        Coordinates  \
0  [H]C([H])([H])[H]  [[tensor(-0.0127), tensor(1.0858), tensor(0.00...   
1       [H]N([H])[H]  [[tensor(-0.0404), tensor(1.0241), tensor(0.06...   
2            [H]O[H]  [[tensor(-0.0344), tensor(0.9775), tensor(0.00...   
3          [H]C#C[H]  [[tensor(0.5995), tensor(0.), tensor(1.)], [te...   
4             [H]C#N  [[tensor(-0.0133), tensor(1.1325), tensor(0.00...   

        HOMO      LUMO       Egap  
0 -10.549854  3.186453  13.736308  
1  -6.993326  2.255824   9.249149  
2  -7.967494  1.869422   9.836916  
3  -7.741639  1.376896   9.118535  
4  -9.806983  0.519737  10.329442  


In [35]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from rdkit import Chem
from rdkit.Chem import AllChem

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.lstm(embedded)
        output = self.fc(output[:, -1, :])
        return output

# Define a custom dataset
class SMILESDataset(Dataset):
    def __init__(self, smiles, egap):
        self.smiles = smiles
        self.egap = egap
        
    def __len__(self):
        return len(self.smiles)
    
    def __getitem__(self, idx):
        return self.smiles[idx], self.egap[idx]

# Define a function to convert SMILES string to tensor
def smiles_to_tensor(smiles):
    mol = Chem.MolFromSmiles(smiles)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2)
    arr = torch.tensor(fp.ToBitString(), dtype=torch.float)
    return arr

# Define hyperparameters
input_size = 2048  # Size of the input tensor (fingerprint)
hidden_size = 128  # Size of the hidden state in LSTM
num_layers = 2  # Number of LSTM layers
output_size = 1  # Size of the output (Egap)

# Create the LSTM model
model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Convert SMILES strings to tensors
smiles_tensor = [smiles_to_tensor(smiles) for smiles in data['SMILES']]
egap_tensor = torch.tensor(data['Egap'].values, dtype=torch.float)

# Create the dataset and dataloader
dataset = SMILESDataset(smiles_tensor, egap_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Train the LSTM model
num_epochs = 10
for epoch in range(num_epochs):
    for smiles, egap in dataloader:
        optimizer.zero_grad()
        output = model(smiles)
        loss = criterion(output, egap.unsqueeze(1))
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

# Save the trained model
torch.save(model.state_dict(), 'lstm_model.pth')


TypeError: new(): invalid data type 'str'

In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

class GNNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GNNConv, self).__init__(aggr='add')
        self.lin = nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        x = self.lin(x)
        return self.propagate(edge_index, x=x)

    def message(self, x_j, edge_index, size):
        row, col = edge_index
        deg = degree(row, size[0], dtype=x_j.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        return x_j * norm.view(-1, 1)

class GNN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNN, self).__init__()
        self.conv1 = GNNConv(in_channels, hidden_channels)
        self.conv2 = GNNConv(hidden_channels, out_channels)
        self.lin = nn.Linear(out_channels, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.lin(x)
        return x

# Create a sample molecule with XYZ coordinates
coords = torch.tensor([
    [-1.2700e-02,  1.0858e+00,  8.0000e-03],
    [ 2.2000e-03, -6.0000e-03,  2.0000e-03],
    [ 1.0117e+00,  1.4638e+00,  3.0000e-04],
    [-5.4080e-01,  1.4475e+00, -8.7660e-01],
    [-5.2380e-01,  1.4379e+00,  9.0640e-01]
])

# Create a sample target value (Egap)
target = torch.tensor([13.736308])

# Create a graph data object
data = Data(x=coords, edge_index=None, y=target)

# Define the GNN model
gnn = GNN(in_channels=3, hidden_channels=64, out_channels=64)

# Forward pass
output = gnn(data)
print("Predicted Egap:", output.item())


AttributeError: 'NoneType' object has no attribute 'device'